### В этой работе:
1. Скачаем [датасет](https://www.manythings.org/anki/rus-eng.zip) англо-русскую пару фраз
2. Обучим seq2seq модель.
3. Поэкспериментируем с различными слоями RNN, GRU и LSTM
4. Оценим качество модели


### Загрузим данные

In [1]:
%matplotlib inline

In [2]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# загрузим данные
!wget https://www.manythings.org/anki/rus-eng.zip

--2025-02-11 19:23:58--  https://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16305013 (16M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  15.55M  32.8MB/s    in 0.5s    

2025-02-11 19:23:58 (32.8 MB/s) - ‘rus-eng.zip’ saved [16305013/16305013]



In [ ]:
# распакуем
import zipfile

with zipfile.ZipFile(r"D:\Progect\RNN\rus-eng.zip", 'r') as zip_ref:
    zip_ref.extractall("D:\Progect\RNN")

In [4]:
# смотрим что внутри, выведем последние 10 строк текста
with open(r"/content/eng-rus.txt", 'r') as file:
    lines = file.readlines()
    for line in lines[-10:]:  # Shows last 10 lines
        print(line.strip())

We need to uphold laws against discrimination — in hiring, and in housing, and in education, and in the criminal justice system. That is what our Constitution and our highest ideals require.	Нам нужно отстаивать законы против дискриминации при найме на работу, в жилищной сфере, в сфере образования и правоохранительной системе. Этого требуют наша Конституция и высшие идеалы.	CC-BY 2.0 (France) Attribution: tatoeba.org #5762728 (BHO) & #6390439 (odexed)
I've heard that you should never date anyone who is less than half your age plus seven. Tom is now 30 years old and Mary is 17. How many years will Tom need to wait until he can start dating Mary?	Я слышал, что никогда не следует встречаться с кем-то вдвое младше вас плюс семь лет. Тому 30 лет, a Мэри 17. Сколько лет Тому нужно ждать до тех пор, пока он сможет начать встречаться с Мэри?	CC-BY 2.0 (France) Attribution: tatoeba.org #10068197 (CK) & #10644473 (notenoughsun)
I do have one final ask of you as your president, the same thing I a

### Выполним предварительную обработку данных

In [6]:
SOS_token = 0   # Start of sequence
EOS_token = 1   # End of sequence

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [7]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427

# Удалим диакритические знаки
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'  # проверяем, что категория символа c не является "Mn" (Mark, Nonspacing), то есть это не диакритический знак.
    )

# проведем предобработку текста
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    # s = re.sub(r"[^а-яА-Я.!?]+", r" ", s)
    return s

In [8]:
# прочитаем текст с парами предложений
# флаг reverse, указывает нужно ли поменять язык местами
# если  reverse=True - меняем порядок предложений в каждой паре
# если reverse=False - оставляем порядок предложений без изменений
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Прочитаем текст построчно
    # разделитель в тексте разный, в моем случаее это CC-BY
    lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')
    lines = [i.split('\tCC-BY', 1)[0] for i in lines]

    # Соеденим каждую строку в пару предложений, проведем нормализацию текста
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Возвращаем список пар слов
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [12]:
# максимальное количество слов в предложении
MAX_LENGTH = 20

# Префиксы для предложений
ENGLISH_PREFIXES = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filter_pair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(ENGLISH_PREFIXES)

def filter_pairs(pairs):
    return [pair for pair in pairs if filter_pair(pair)]

In [13]:
# функция подготовки данных для машинного перевода
# lang1 и lang2 - коды языков (например, 'eng' для английского, 'rus' для русского)
# reverse=False - направление перевода
def prepareData(lang1, lang2, reverse=False):

    # выведим количество прочитанных пар предложений
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))

    # выведим количество пар предложений после фильтрации
    pairs = filter_pairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])

    # выводит имя языка и количество уникальных слов
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

# параметры машинного перевода: 'eng' - английский язык, 'rus' - русский язык
# True - перевод будет осуществляться с русского на английский
input_lang, output_lang, pairs = prepareData('eng', 'rus', True)

# проверим корректность работы функции
print(random.choice(pairs))

Reading lines...
Read 496059 sentence pairs
Trimmed to 5113 sentence pairs
Counting words...
Counted words:
rus 4792
eng 2446
['она истинно обворожительнеишая молодая леди .', 'she is a most charming young lady indeed .']


Прочитали 496 059 пар предложений   
После фильтрации осталось 5113 пар предложений   
Посчитали количество уникальных пар предложений:   
для русского языка - 4792   
для английского языка - 2446   

При этом, если мы изменим параметр MAX_LENGTH больше чем 20, тогда получим большее число уникальных предложений, если уменьшим, то, соответственно получим меньшее количество уникальных предложений

-------------------------------------
Если на этом этапе в результате предобработки мы увидели корректный текст на двух разных языках (в нашем варианте 'eng'и 'rus'), тогда можно приступать к следующему шагу, к созданию архитектуры модели и её обучению.    
Если до этого момента на выходе получили непонятный текст или ошибку, то следует вернуться к повторной обработке данных, пока не получим корректный результат.

### Архитектура GRU-модели c одним слоем

In [14]:
# Encoder
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input_seq, hidden):
        embedded = self.embedding(input_seq).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [15]:
# Decoder
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [16]:
# преобразуем предложения в список индексов и слов
def indexes_from_sentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

# преобразуем предложение в тензор PyTorch
def tensor_from_sentence(lang, sentence):
    indexes = indexes_from_sentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

# создадим тензоры для пары предложений
def tensors_from_pair(pair):
    input_tensor = tensor_from_sentence(input_lang, pair[0])
    target_tensor = tensor_from_sentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [22]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.init_hidden()

     # Инициализация
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    # Кодирование входного предложения
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    # декодирование
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Обучение с teacher forcing или без него
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    # Обратное распространение ошибки и обновление моделей
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [21]:
import time
import math

def as_minutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def time_since(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (as_minutes(s), as_minutes(rs))

In [19]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def show_plot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

#### Обучение модели

In [23]:
# обучение модели
def train_iterations(
    encoder,           # модель-кодировщик
    decoder,          # модель-декодировщик
    n_iters,          # количество итераций
    print_every=1000, # интервал печати статистики
    plot_every=100,   # интервал обновления графика
    learning_rate=0.01 # скорость обучения
):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    # Инициализация оптимизаторов
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    # Подготовка данных для обучения
    training_pairs = [tensors_from_pair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    # Цикл обучения
    for iter in range(1, n_iters + 1):
        # Получение пары для обучения
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        # Обучение на текущей паре
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)

        # Накопление потерь
        print_loss_total += loss
        plot_loss_total += loss

         # Печать статистики
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (time_since(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        # Обновление графика
        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    show_plot(plot_losses)

In [24]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        # Подготовка входных данных
        input_tensor = tensor_from_sentence(input_lang, sentence)
        input_length = input_tensor.size()[0]

        # Инициализация кодировщика
        encoder_hidden = encoder.init_hidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        # Кодирование входного предложения
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        # Инициализация декодировщика
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden
        decoded_words = []

        # Генерация перевода
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [25]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
# зададим параметр скрытого состояния, это влияет на способность модели запоминать контекст
# больший размер даёт больше возможностей для хранения информации, но увеличивает вычислительные затраты
hidden_size = 256
encoderRNN = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoderRNN = DecoderRNN(hidden_size, output_lang.n_words).to(device)

# 75000 - количество итераций обучения
# print_every=5000 - каждые 5000 итераций будет выводиться статистика обучения
train_iterations(encoderRNN, decoderRNN, 75000, print_every=5000)

1m 17s (- 18m 11s) (5000 6%) 3.0949
2m 26s (- 15m 54s) (10000 13%) 2.5096
3m 37s (- 14m 29s) (15000 20%) 2.0018
4m 47s (- 13m 9s) (20000 26%) 1.6168
5m 57s (- 11m 55s) (25000 33%) 1.2623
7m 8s (- 10m 43s) (30000 40%) 0.9882
8m 19s (- 9m 30s) (35000 46%) 0.7511
9m 30s (- 8m 19s) (40000 53%) 0.5693
10m 42s (- 7m 8s) (45000 60%) 0.4360
11m 53s (- 5m 56s) (50000 66%) 0.3229
13m 4s (- 4m 45s) (55000 73%) 0.2308
14m 16s (- 3m 34s) (60000 80%) 0.1619
15m 27s (- 2m 22s) (65000 86%) 0.1282
16m 38s (- 1m 11s) (70000 93%) 0.0975
17m 50s (- 0m 0s) (75000 100%) 0.0768


#### Оценка модели

In [ ]:
evaluateRandomly(encoderRNN, decoderRNN)

> ты неодета .
= you aren't dressed .
< you aren't dressed . <EOS>

> он такого же роста, как мои отец .
= he is as tall as my father .
< he is as tall as my father . <EOS>

> они летчицы .
= they are pilots .
< they are pilots . <EOS>

> я под одеялом .
= i am under the blanket .
< i am under the blanket . <EOS>

> она не так молода, как выглядит .
= she is not as young as she looks .
< she is not as young as she looks . <EOS>

> его там нет .
= he is not there .
< he is not there . <EOS>

> его все уважают .
= he is respected by everyone .
< he is respected by everyone . <EOS>

> еи за двадцать .
= she is over twenty .
< she is over twenty . <EOS>

> он беден, но счастлив .
= he is poor, but happy .
< he is poor, but happy . <EOS>

> она показала мне свои альбом .
= she showed me her album .
< she showed me her album . <EOS>



### Добавим RNN слой в encoder и decoder

#### Архитектура модели

In [27]:
class EncoderGRU_RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderGRU_RNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)

        # Первый GRU слой
        self.gru1 = nn.GRU(hidden_size, hidden_size)

        # Добавлен второй RNN слой
        self.gru2 = nn.RNN(hidden_size, hidden_size)

    def forward(self, input_seq, hidden):
        embedded = self.embedding(input_seq).view(1, 1, -1)
        output = embedded

        # Пропуск через первый GRU слой
        output, hidden1 = self.gru1(output, hidden)

        # Пропуск через второй RNN слой
        output, hidden2 = self.gru2(output, hidden1)

        return output, hidden2

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [28]:
class DecoderGRU_RNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderGRU_RNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)

        # Первый GRU слой
        self.gru1 = nn.GRU(hidden_size, hidden_size)

        # Добавлен второй RNN слой
        self.gru2 = nn.RNN(hidden_size, hidden_size)

        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)

        # Пропуск через первый GRU слой
        output, hidden1 = self.gru1(output, hidden)

        # Пропуск через второй RNN слой
        output, hidden2 = self.gru2(output, hidden1)

        output = self.softmax(self.out(output[0]))
        return output, hidden2

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

#### Обучим модель

In [29]:
hidden_size = 256
encoderGRU_RNN = EncoderGRU_RNN(input_lang.n_words, hidden_size).to(device)
decoderGRU_RNN = DecoderGRU_RNN(hidden_size, output_lang.n_words).to(device)

train_iterations(encoderGRU_RNN, decoderGRU_RNN, 75000, print_every=5000)

1m 57s (- 27m 19s) (5000 6%) 3.3869
3m 40s (- 23m 50s) (10000 13%) 3.0830
5m 21s (- 21m 26s) (15000 20%) 3.0266
7m 7s (- 19m 34s) (20000 26%) 2.9329
8m 49s (- 17m 39s) (25000 33%) 2.9363
10m 32s (- 15m 49s) (30000 40%) 2.9706
12m 20s (- 14m 6s) (35000 46%) 2.9119
14m 4s (- 12m 18s) (40000 53%) 2.8476
15m 47s (- 10m 31s) (45000 60%) 2.8544
17m 27s (- 8m 43s) (50000 66%) 2.8446
19m 8s (- 6m 57s) (55000 73%) 2.8819
20m 47s (- 5m 11s) (60000 80%) 2.8957
22m 26s (- 3m 27s) (65000 86%) 2.9330
24m 6s (- 1m 43s) (70000 93%) 2.9857
25m 45s (- 0m 0s) (75000 100%) 2.9260


In [30]:
evaluateRandomly(encoderGRU_RNN, decoderGRU_RNN)

> она очень стеснительная девочка .
= she is a very shy girl .
< she is going to . <EOS>

> она беспокоится о твоем здоровье .
= she is anxious about your health .
< she is no to <EOS>

> она держит ложку левои рукои .
= she is holding a ladle with her left hand .
< she is not to . <EOS>

> только тебе я могу доверять .
= you are the only one i can trust .
< they aren't used to . <EOS>

> мы с неи помолвлены .
= i am engaged to her .
< she is going to . <EOS>

> она страдает от низкого кровяного давления .
= she suffers from low blood pressure .
< she is going to . <EOS>

> ты очень храбрыи .
= you are very brave .
< you are a . <EOS>

> она уже не та, какои была десять лет назад .
= she is not what she was ten years ago .
< she is going to . <EOS>

> я ищу подарок маме .
= i am looking for a present for my mother .
< you are a . <EOS>

> они изучают проблему .
= they are looking into the problem .
< they aren't used to . <EOS>



### Заменим на LSTM

#### Архитектура модели

In [34]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)

        # Заменены на LSTM
        self.lstm1 = nn.LSTM(hidden_size, hidden_size)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input_seq, hidden):
        embedded = self.embedding(input_seq).view(1, 1, -1)
        output = embedded

        # Обработка через первичный LSTM
        output, hidden1 = self.lstm1(output, hidden)

        # Обработка через вторичный LSTM
        output, hidden2 = self.lstm2(output, hidden1)

        return output, hidden2
    def init_hidden(self):
        return (
            torch.zeros(1, 1, self.hidden_size, device=device),
            torch.zeros(1, 1, self.hidden_size, device=device)
        )

In [37]:
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)

        # Заменены на LSTM
        self.lstm1 = nn.LSTM(hidden_size, hidden_size)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size)

        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)

        # Обработка через первичный LSTM
        output, hidden1 = self.lstm1(output, hidden)

        # Обработка через вторичный LSTM
        output, hidden2 = self.lstm2(output, hidden1)

        output = self.softmax(self.out(output[0]))
        return output, hidden2

#### Обучим модель

In [38]:
hidden_size = 256
encoderLSTM = EncoderLSTM(input_lang.n_words, hidden_size).to(device)
decoderLSTM = DecoderLSTM(hidden_size, output_lang.n_words).to(device)

train_iterations(encoderLSTM, decoderLSTM, 75000, print_every=5000)

2m 9s (- 30m 16s) (5000 6%) 3.4496
4m 11s (- 27m 14s) (10000 13%) 3.1889
6m 13s (- 24m 54s) (15000 20%) 3.1088
8m 15s (- 22m 42s) (20000 26%) 2.8794
10m 18s (- 20m 37s) (25000 33%) 2.7006
12m 22s (- 18m 33s) (30000 40%) 2.5756
14m 26s (- 16m 30s) (35000 46%) 2.5181
16m 33s (- 14m 29s) (40000 53%) 2.4540
18m 39s (- 12m 26s) (45000 60%) 2.3876
20m 46s (- 10m 23s) (50000 66%) 2.3070
22m 54s (- 8m 19s) (55000 73%) 2.2122
25m 2s (- 6m 15s) (60000 80%) 2.1535
27m 11s (- 4m 10s) (65000 86%) 2.0386
29m 19s (- 2m 5s) (70000 93%) 1.9723
31m 28s (- 0m 0s) (75000 100%) 1.8676


In [42]:
evaluateRandomly(encoderLSTM, decoderLSTM)

> мы находимся под его командованием .
= we are under his command .
< we are looking forward . <EOS>

> он катается на коньках .
= he is skating .
< he is good in the . . <EOS>

> я рад слышать эти известия .
= i am glad to hear the news .
< i am afraid that i will you . . <EOS>

> он вооружен до зубов .
= he is armed to the teeth .
< he is going to the . . <EOS>

> только тебе я могу доверять .
= you are the only one i can trust .
< you are the last person that i expected . <EOS>

> они обеспеченные люди .
= they are well off .
< they are all . <EOS>

> с ним тяжело иметь дело .
= he is hard to deal with .
< he is too young at the . . <EOS>

> вы ведь нам не все рассказываете ?
= you aren't telling us everything, are you ?
< you aren't mad at me, are you ? <EOS>

> я до конца сентября работаю на полную ставку в книжном магазине .
= i am working full-time at a bookshop until the end of september .
< i am looking to be a in the the the the the the the . <EOS>

> она такого же роста, как

### Итоги:

In [41]:
import pandas as pd
data = {
    'time': ['17m 50s', '25m 45s', '31m 28s'],
    'loss_avg': [0.0768, 2.9260, 1.8676]
}
df = pd.DataFrame(data, index=['GRU', 'GRU_RNN', 'LSTM'])
df


,time,loss_avg
GRU,17m 50s,0.0768
GRU_RNN,25m 45s,2.9260
LSTM,31m 28s,1.8676


**Напомню исходные данные:**
1. GRU: состоит из одного слоя
2. GRU_RNN: состоит из одного слоя GRU и одного слоя RNN
3. LSTM: состоит из двух слоев LSTM

**Анализ результатов:**

*GRU:*   
* Показал наилучшие результаты по loss (0.0768)
* Самая быстрая скорость обучения (17м 50с)
* Это ожидаемо, так как GRU имеет упрощенную архитектуру

*GRU_RNN:*   
* Наихудший loss (2.9260)
* Средняя скорость обучения (25м 45с)
* Комбинированная архитектура не улучшила результаты

*LSTM:*
* Средний loss (1.8676)
* Самая долгая тренировка (31м 28с)
* Более сложная архитектура требует больше времени

**Выводы:**

GRU показал оптимальный баланс между скоростью и качеством обучения.
Возможно при увеличении объема данных LSTM получила бы преимущество, т.к. loss снижался с 3.4496 до 1.8676. Я считаю, что в этом есть потенциал увеличения качества модели, т.к. я намеренно усложнил модель двумя слоями LSTM, против одного слоя GRU.   

Используем GRU для небольших наборов данных, используем LSTM при наличии достаточного объема данных и вычислительных ресурсов, комбинированные архитектуры в моем случае не показали явных преимуществ.